<a href="https://colab.research.google.com/github/ujwalreddy18/Twitter-spam-detection/blob/main/NLP_N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
df = pd.read_csv("/content/amazon_alexa (1).tsv", sep="\t")

df = df[["verified_reviews", "feedback"]]
df.columns = ["text", "label"]
df.dropna(inplace=True)

df.head()


,text,label
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


In [ ]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('punkt_tab')

lemm = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z ]", " ", text)
    tokens = nltk.word_tokenize(text)
    tokens = [lemm.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

df["cleaned"] = df["text"].apply(clean_text)
df.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,text,label,cleaned
0,Love my Echo!,1,love echo
1,Loved it!,1,loved
2,"Sometimes while playing a game, you can answer...",1,sometimes playing game answer question correct...
3,I have had a lot of fun with this thing. My 4 ...,1,lot fun thing yr old learns dinosaur control l...
4,Music,1,music


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df["cleaned"], df["label"], test_size=0.2, random_state=42
)


In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [ ]:
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

nb_pred = nb.predict(X_test_tfidf)
print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_pred))


Naive Bayes Accuracy: 0.9


In [ ]:
lr = LogisticRegression(max_iter=300)
lr.fit(X_train_tfidf, y_train)

lr_pred = lr.predict(X_test_tfidf)
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_pred))


Logistic Regression Accuracy: 0.9015873015873016


In [ ]:
svm = LinearSVC()
svm.fit(X_train_tfidf, y_train)

svm_pred = svm.predict(X_test_tfidf)
print("SVM Accuracy:", accuracy_score(y_test, svm_pred))


SVM Accuracy: 0.9174603174603174


In [ ]:
print("Classification Report (SVM):")
print(classification_report(y_test, svm_pred))

print("Confusion Matrix (SVM):")
print(confusion_matrix(y_test, svm_pred))


Classification Report (SVM):
              precision    recall  f1-score   support

           0       0.72      0.29      0.41        63
           1       0.93      0.99      0.96       567

    accuracy                           0.92       630
   macro avg       0.82      0.64      0.68       630
weighted avg       0.91      0.92      0.90       630

Confusion Matrix (SVM):
[[ 18  45]
 [  7 560]]


In [ ]:
# ================================
# BLOCK 11 — Word2Vec-Style LSTM
# ================================

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout

# Tokenizer
tokenizer_lstm = Tokenizer(num_words=20000, oov_token="<UNK>")
tokenizer_lstm.fit_on_texts(df["cleaned"])

max_len = 60

# Convert text → sequences
X_train_seq = tokenizer_lstm.texts_to_sequences(X_train)
X_test_seq  = tokenizer_lstm.texts_to_sequences(X_test)

# Padding
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad  = pad_sequences(X_test_seq,  maxlen=max_len, padding='post')

vocab_size = min(20000, len(tokenizer_lstm.word_index) + 1)

# Build LSTM model
model_w2v = Sequential([
    Embedding(vocab_size, 100, trainable=True),
    Bidirectional(LSTM(64)),
    Dropout(0.4),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Build model explicitly
model_w2v.build(input_shape=(None, max_len))
model_w2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_w2v.summary()

# Train
history_w2v = model_w2v.fit(
    X_train_pad, y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=64,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=2, restore_best_weights=True
        )
    ]
)

# Evaluate
y_pred_prob_w2v = model_w2v.predict(X_test_pad)
y_pred_w2v = (y_pred_prob_w2v >= 0.5).astype(int)

print("Word2Vec-LSTM Accuracy:", accuracy_score(y_test, y_pred_w2v))
print("\nClassification Report:\n", classification_report(y_test, y_pred_w2v))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_w2v))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 60, 100)        │       349,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 437,941 (1.67 MB)

 Trainable params: 437,941 (1.67 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 21s 312ms/step - accuracy: 0.8208 - loss: 0.4763 - val_accuracy: 0.9484 - val_loss: 0.2246
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 290ms/step - accuracy: 0.9200 - loss: 0.2756 - val_accuracy: 0.9484 - val_loss: 0.1853
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 268ms/step - accuracy: 0.9175 - loss: 0.2612 - val_accuracy: 0.9484 - val_loss: 0.1497
Epoch 4/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 14s 386ms/step - accuracy: 0.9460 - loss: 0.1576 - val_accuracy: 0.9524 - val_loss: 0.1651
Epoch 5/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 284ms/step - accuracy: 0.9596 - loss: 0.1035 - val_accuracy: 0.9524 - val_loss: 0.1387
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step
Word2Vec-LSTM Accuracy: 0.9253968253968254

Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.44      0.54        63
           1       0.94      0.98      0.96       567

    accuracy                           0.93       630
   macro avg       0.82      0.71 

In [ ]:
# ================================
# BLOCK 12 — BERT → LSTM (PyTorch)
# ================================

import torch
import torch.nn as nn
from transformers import DistilBertTokenizerFast, DistilBertModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load PyTorch DistilBERT
bert_tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
bert_model = DistilBertModel.from_pretrained("distilbert-base-uncased").to(device)
bert_model.eval()

max_len = 64

def encode_bert(text_list):
    encodings = bert_tokenizer(
        text_list,
        truncation=True,
        padding="max_length",
        max_length=max_len,
        return_tensors="pt"
    )
    return encodings["input_ids"], encodings["attention_mask"]

# Encode train + test
input_ids_train, mask_train = encode_bert(X_train.tolist())
input_ids_test,  mask_test  = encode_bert(X_test.tolist())

input_ids_train = input_ids_train.to(device)
mask_train = mask_train.to(device)
input_ids_test = input_ids_test.to(device)
mask_test = mask_test.to(device)

# Extract BERT embeddings
def get_bert_embeddings(input_ids, masks):
    with torch.no_grad():
        outputs = bert_model(input_ids=input_ids, attention_mask=masks)
        return outputs.last_hidden_state  # (batch, seq_len, hidden=768)

X_train_bert = get_bert_embeddings(input_ids_train, mask_train)
X_test_bert  = get_bert_embeddings(input_ids_test,  mask_test)

# ============================
# LSTM on top of BERT embeddings
# ============================

class BertLSTM(nn.Module):
    def __init__(self):
        super(BertLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size=768,
            hidden_size=128,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(0.4)
        self.fc1 = nn.Linear(256, 64)
        self.dropout2 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last = lstm_out[:, -1, :]
        x = self.dropout(last)
        x = torch.relu(self.fc1(x))
        x = self.dropout2(x)
        x = torch.sigmoid(self.fc2(x))
        return x

model_bert_lstm = BertLSTM().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_bert_lstm.parameters(), lr=2e-4)

# Convert targets to tensor
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)

# Training loop (3 epochs)
for epoch in range(3):
    model_bert_lstm.train()
    optimizer.zero_grad()

    outputs = model_bert_lstm(X_train_bert)
    outputs = outputs.view(-1)

    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1} Loss:", loss.item())

# Evaluation
model_bert_lstm.eval()
with torch.no_grad():
    test_outputs = model_bert_lstm(X_test_bert).view(-1)
    y_pred_bert = (test_outputs >= 0.5).cpu().numpy().astype(int)

print("BERT-LSTM Accuracy:", accuracy_score(y_test, y_pred_bert))
print(classification_report(y_test, y_pred_bert))
print(confusion_matrix(y_test, y_pred_bert))

# Save confusion matrix for comparison
cm_bert = confusion_matrix(y_test, y_pred_bert)

Epoch 1 Loss: 0.703804612159729
Epoch 2 Loss: 0.6906123161315918
Epoch 3 Loss: 0.676768958568573
BERT-LSTM Accuracy: 0.9
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        63
           1       0.90      1.00      0.95       567

    accuracy                           0.90       630
   macro avg       0.45      0.50      0.47       630
weighted avg       0.81      0.90      0.85       630

[[  0  63]
 [  0 567]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
def predict_review(text):
    text_cleaned = clean_text(text)
    text_tfidf = tfidf.transform([text_cleaned])
    pred = svm.predict(text_tfidf)[0]

    if pred == 1:
        return "Positive Review 😊"
    else:
        return "Negative Review 😠"

user_input = input("Enter an Alexa review: ")
print("Prediction:", predict_review(user_input))
